In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-06-26"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
850,2024-06-26,Mundo Amistoso Internacional,14:00,Montenegro,Bahamas,3.44,1.28,162.5,1.80,1.90,0.0,0.00,0.00,IcrkZNvQ,0.290698,0.781250,0.555556,0.526316,0.071948,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,57,0,6.20,0.00,0.000,0.000,0.647182,0.038222,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
851,2024-06-26,Europa Campeonato Das Nações Europeias Menores,14:30,Andorra,Gibraltar,1.29,3.37,145.5,1.83,1.87,0.0,0.00,0.00,0tnJ2w0c,0.775194,0.296736,0.546448,0.534759,0.071930,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,63,89,1.04,12.50,0.000,0.000,0.631237,0.015289,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
852,2024-06-26,Europa Campeonato Das Nações Europeias Menores,12:00,Malta,São Marino,1.10,6.25,147.5,1.87,1.83,0.0,0.00,0.00,pY3Wvz8j,0.909091,0.160000,0.534759,0.546448,0.069091,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,81,68,1.02,9.10,0.000,0.000,0.990912,0.015289,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
853,2024-06-26,Colômbia Lbp,22:10,Cimarrones del Choco,Piratas de Bogota,1.64,2.06,160.5,1.85,1.85,0.0,0.00,0.00,Ma2fN2Ba,0.609756,0.485437,0.540541,0.540541,0.095193,245.838,146.071820,0.594179,1.2,1.643168,1.369306,373.68,3.082,1.869618,0.606625,-21.0,269.776,49.756216,0.184435,1.2,1.643168,1.369306,292.60,3.060,0.563604,0.184184,-24.0,72,95,5.19,3.08,202.364,256.326,0.160532,0.000000,NaN,-2.07,-0.414,-1.545894,0.000000,0.0,0.000000,-2.53,-0.506,-2.094862,0.000000,0.0,0.000000
854,2024-06-26,Colômbia Lbp,20:10,Titanes de Barranquilla,Motilones del Norte,1.34,3.08,156.5,1.85,1.85,0.0,0.00,0.00,0pEoPOtm,0.746269,0.324675,0.540541,0.540541,0.070944,117.898,33.682828,0.285695,2.4,1.341641,0.559017,89.08,1.386,0.226340,0.163305,79.0,164.658,34.831480,0.211538,1.2,1.643168,1.369306,169.00,2.180,0.824227,0.378086,-22.0,68,52,1.31,3.25,88.754,164.658,0.556727,0.000000,NaN,-1.48,-0.296,-1.148649,0.833695,0.8,-0.033695,-1.43,-0.286,-7.272727,0.554667,0.4,-0.154667
855,2024-06-26,Vietnã Vba,09:30,Nhatrang Dolphins,Hanoi Buffaloes,2.95,1.37,188.5,1.82,1.88,0.0,0.00,0.00,E5yz1sPq,0.338983,0.729927,0.549451,0.531915,0.068910,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0,0,0.00,0.00,0.000,0.000,0.517236,0.022933,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
856,2024-06-26,Venezuela Superliga,20:00,Cocodrilos,Guaiqueries,1.33,3.12,156.5,1.82,1.88,0.0,0.00,0.00,Cz82FusQ,0.751880,0.320513,0.549451,0.531915,0.072393,117.532,31.273866,0.266088,3.0,0.000000,0.000000,117.12,1.474,0.390871,0.265177,30.0,159.786,41.139187,0.257464,3.0,0.000000,0.000000,194.34,1.970,0.432261,0.219422,45.0,96,82,1.22,2.37,119.990,148.592,0.568863,0.022933,NaN,1.51,0.302,1.092715,0.000000,0.0,0.000000,0.89,0.178,11.910112,0.000000,0.0,0.000000
857,2024-06-26,Venezuela Superlig

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

Sem jogos
